# QCELS Circuit Implementation

- Task implement the circuit form of qcels given in the scipy implementation
- Build a Hadamard Test circuit
- Use `ControlledPauliExpBox` and  `StatePreperationBox` (or whatever ansatz you want)

pytket circuit construction - https://cqcl.github.io/pytket/manual/manual_circuit.html

In [ ]:
! pip install pytket pytket-qulacs

In [17]:
from circuits import ControlledPauliExpBox
from pytket.circuit import Circuit, Qubit, StatePreparationBox
from hamiltonians import ising_model

In [18]:
n_qubits = 3
h = 1
j = 1
ising_ham = ising_model(n_qubits, h, 1)

`ising_ham._dict` : {keys: pauls, values: coeffs}

In [19]:
ising_ham._dict

{(Zq[0], Zq[1]): 1.00000000000000,
 (Zq[1], Zq[2]): 1.00000000000000,
 (Xq[0]): 1.00000000000000,
 (Xq[1]): 1.00000000000000,
 (Xq[2]): 1.00000000000000}

- Controlled pauli example
- The coeff will need to be modified by the size of the trotter step $\theta = 2 h_a \frac{t}{n}$
- $\theta$ is in units of $\pi$

In [20]:
from dataclasses import dataclass
@dataclass
class PauliTerm:
    pauli: str
    coeff: float

pauli_coeff_list = [PauliTerm(pauli, coeff) for pauli, coeff in ising_ham._dict.items()]

- Make this into a function for each term

In [21]:
term_0 = pauli_coeff_list[0]
pauli_list = list(term_0.pauli.map.values())
qubit_ind_list = list(term_0.pauli.map.keys())
c_paulibox = ControlledPauliExpBox(pauli_list, qubit_ind_list, term_0.coeff)

In [22]:
from pytket.circuit.display import render_circuit_jupyter
render_circuit_jupyter(c_paulibox.get_circuit())

- Circuit composition example

In [23]:
circ =  Circuit(n_qubits)
a = circ.add_q_register('a', 1) # Add ancilla register
circ.qubits

[a[0], q[0], q[1], q[2]]

In [24]:
circ.H(Qubit('a',0)) #Hadamard test
circ.Ry(0.5, Qubit('q',0)).Ry(0.5, Qubit('q',1)).Ry(0.2, Qubit('q',2)) #Random inital state
circ.add_circbox(c_paulibox, [a[0], *qubit_ind_list])
circ.H(Qubit('a',0)) #Hadamard test

[H a[0]; Ry(0.5) q[0]; Ry(0.5) q[1]; Ry(0.2) q[2]; CircBox a[0], q[0], q[1]; H a[0]; ]

In [25]:
render_circuit_jupyter(circ)
# hover over the circut box

- Use the thic circuit to build the hadmard test

pytket running on backends - https://cqcl.github.io/pytket/manual/manual_backend.html

In [26]:
from pytket.extensions.qulacs import QulacsBackend

In [27]:
backend = QulacsBackend()
compiled_circ = backend.get_compiled_circuit(circ)
render_circuit_jupyter(compiled_circ) # Boxes are decompoed on compilation
print('Circuit depth: ',compiled_circ.depth())

Circuit depth:  12


In [28]:
handle = backend.process_circuit(compiled_circ,)
dist = backend.get_result(handle).get_distribution()
dist

{(0, 0, 0, 0): 0.11306356214843423,
 (0, 0, 0, 1): 0.01193643785156578,
 (0, 0, 1, 0): 0.11306356214843426,
 (0, 0, 1, 1): 0.011936437851565795,
 (0, 1, 0, 0): 0.11306356214843427,
 (0, 1, 0, 1): 0.01193643785156579,
 (0, 1, 1, 0): 0.11306356214843416,
 (0, 1, 1, 1): 0.011936437851565774,
 (1, 0, 0, 0): 0.11306356214843426,
 (1, 0, 0, 1): 0.011936437851565788,
 (1, 0, 1, 0): 0.11306356214843417,
 (1, 0, 1, 1): 0.011936437851565783,
 (1, 1, 0, 0): 0.11306356214843423,
 (1, 1, 0, 1): 0.01193643785156578,
 (1, 1, 1, 0): 0.11306356214843419,
 (1, 1, 1, 1): 0.011936437851565784}

- Concatenate (sum) the first bit ancilla results for zero and 1 and perform the hadamard test
- Generate the plots using the function `f_theta` from `qcels.ipynb`
- compare the trotter error (number of slices)

Hint - You will need to make a trotter step from a series of controlled pauli gadgets. You can then further wrap up all the controlled pauli exp box circuirs with a `Circbox` for a single trotter step. This will make your code easier to use.